**Importing Useful Libraries**

In [278]:
import cv2
import numpy as np
import math
import sys
import xlrd
from skimage.measure import structural_similarity as ssim
from scipy.stats.stats import pearsonr

**Loading Images from DIBR Dataset**

In [279]:
images=[]
oimages=[]
for i in range(1,85,1):
    images.append(cv2.imread("./DIBR_Data/"+str(i)+".png",0));

**Loading Subjective score of images**

In [280]:
loc = ("./DMOS_DIBR.xls"); 
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 
dmos=[]
for i in range(0,84,1):
    dmos.append(sheet.cell_value(i, 0)) 

**Counting the number of Black pixels in images with a threshold value of 15.**

In [283]:
geo_distortion=[]
count=[]
for img in images:
    ii=np.empty((len(img),len(img[0])),dtype=np.float64);
    c=5
    for i in range(0,len(img),1):
        for j in range(0,len(img[0]),1):
            if img[i][j]>15:
                ii[i][j]=255.0;
            else:
                ii[i][j]<=15.0;
                c=c+1;
    geo_distortion.append(ii);
    count.append(c);

**Calculating variance in images using laplacian.**

In [284]:
var_images=[]
var_geo_images=[]
for i in range(0,84,1):
    var_images.append(cv2.Laplacian(images[i], cv2.CV_64F).var())
    var_geo_images.append(cv2.Laplacian(geo_distortion[i], cv2.CV_64F).var())

**Normalizing value of Variance**

In [285]:
norm_images = [float(i)/sum(var_images) for i in var_images]
norm_geo_images = [float(i)/sum(var_geo_images) for i in var_geo_images]

In [286]:
print("PLCC Value obtained by using area covered by black pixels in images\n"+str(abs(pearsonr(count,dmos)[0])))

PLCC Value obtained by using area covered by black pixels in images
0.635551143925


**Images contains blurred region around the edges of object so adjacent columns in images contains almost same pixel value.**

In [287]:
column_count=[]
for img in images:
    c=0
    for j in range(0,len(img[0])-1,1):
        if np.all(np.abs(np.subtract(img[:,j],img[:,j+1]))<25):
            c=c+1
    column_count.append(c)

**Normalizing value of count of black pixels and count of matching columns**

In [288]:
norm_count = [float(i)/sum(count) for i in count]
norm_column_count = [float(i)/sum(column_count) for i in column_count]

# Combining count of black pixels, count of matching columns and variance of image to give PLCC
norm_combine = [5*norm_count[i]+norm_column_count[i]-0.8*norm_images[i] for i in range(0,84,1)]

print("PLCC using 3 attributes of Image: "+str(abs(pearsonr(norm_combine,dmos)[0])))

PLCC using 3 attributes of Image: 0.656470498142


**Applying Gaussian Filter on images and extracting edges and other high frequency information from images.**

In [289]:
gaussian_images=[]
for i in range(0,84,1):
    blur = cv2.GaussianBlur(images[i],(3,3),1.25)
    blur_distorted= cv2.GaussianBlur(geo_distortion[i],(3,3),1.25)
    gaussian_images.append(cv2.subtract(images[i],blur));
    gaussian_images_distorted.append(cv2.subtract(geo_distortion[i],blur_distorted));

In [290]:
gaussian_images_sub=[]
sum_gaussian_sub_images=[]
for i in range(0,84,1):
    x=cv2.subtract(gaussian_images[i],gaussian_images_distorted[i])
    gaussian_images_sub.append(x);
    sum_gaussian_sub_images.append(x.sum());

In [291]:
norm_sum_gaussian_sub_images=[float(i)/sum(sum_gaussian_sub_images) for i in sum_gaussian_sub_images]

norm_combination = [5*norm_count[i]+0.8*norm_column_count[i]-1.2*norm_images[i]+18*norm_sum_gaussian_sub_images[i] for i in range(0,84,1)]

print("PLCC using 4 attributes of Image: "+str(abs(pearsonr(norm_combination,dmos)[0])))

PLCC using 4 attributes of Image: 0.666513938545
